In [1]:
import numpy as np
import tensorflow as tf

tf.__version__

'2.1.0'

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

# Dataset

In [3]:
from tensorflow.keras.datasets import reuters

max_words  = 400
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words, test_split=0.2)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

Loading data...
8982 train sequences
2246 test sequences
46 classes


## Sequence padding

In [4]:
from tensorflow.keras.preprocessing import sequence

maxlen = 200

print('Pad sequences (samples x time)')
x_train_padded = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test_padded  = sequence.pad_sequences(x_test,  maxlen=maxlen)
print('x_train        shape:', x_train.shape)
print('x_test         shape:', x_test.shape)
print('x_train_padded shape:', x_train_padded.shape)
print('x_test_padded  shape:', x_test_padded.shape)

Pad sequences (samples x time)
x_train        shape: (8982,)
x_test         shape: (2246,)
x_train_padded shape: (8982, 200)
x_test_padded  shape: (2246, 200)


# Model

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Dense, Conv1D, MaxPool1D, GlobalMaxPooling1D, BatchNormalization
from tensorflow.keras.layers import ReLU

In [24]:
embeddings_dim = 200

model = Sequential()

model.add( Embedding(max_words , embeddings_dim , input_length=maxlen, trainable=True)  )
model.add( Dropout(0.6))
model.add( Conv1D(filters=256, kernel_size=(11,), strides=3, padding='same', activation=None) )
model.add( BatchNormalization() )
model.add( ReLU() )
model.add( Conv1D(filters=128, kernel_size=(7,), strides=2, padding='same', activation=None) )
model.add( BatchNormalization() )
model.add( ReLU() )
model.add( MaxPool1D(pool_size=2) )
model.add( Conv1D(filters=128, kernel_size=(5,), strides=2, padding='same', activation=None) )
model.add( BatchNormalization() )
model.add( ReLU() )
model.add( GlobalMaxPooling1D() )
model.add( Dense(100, activation='tanh') )
model.add( Dropout(0.6))
model.add( Dense(num_classes, activation='softmax') )

In [25]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 200, 200)          80000     
_________________________________________________________________
dropout_14 (Dropout)         (None, 200, 200)          0         
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 67, 256)           563456    
_________________________________________________________________
batch_normalization_14 (Batc (None, 67, 256)           1024      
_________________________________________________________________
re_lu_21 (ReLU)              (None, 67, 256)           0         
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 34, 128)           229504    
_________________________________________________________________
batch_normalization_15 (Batc (None, 34, 128)          

In [26]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [27]:
model.fit(x_train_padded, y_train, batch_size=32, epochs=30, verbose=1, validation_data=(x_test_padded, y_test) )

Train on 8982 samples, validate on 2246 samples
Epoch 1/30
8982/8982 [==============================] - 4s 478us/sample - loss: 2.2198 - accuracy: 0.4748 - val_loss: 2.9008 - val_accuracy: 0.3330
Epoch 2/30
8982/8982 [==============================] - 3s 389us/sample - loss: 1.6688 - accuracy: 0.5961 - val_loss: 1.5634 - val_accuracy: 0.6140
Epoch 3/30
8982/8982 [==============================] - 4s 391us/sample - loss: 1.4610 - accuracy: 0.6531 - val_loss: 1.3228 - val_accuracy: 0.6745
Epoch 4/30
8982/8982 [==============================] - 3s 388us/sample - loss: 1.3118 - accuracy: 0.6860 - val_loss: 1.2724 - val_accuracy: 0.6870
Epoch 5/30
8982/8982 [==============================] - 3s 382us/sample - loss: 1.2187 - accuracy: 0.7069 - val_loss: 1.2697 - val_accuracy: 0.6906
Epoch 6/30
8982/8982 [==============================] - 3s 387us/sample - loss: 1.1352 - accuracy: 0.7240 - val_loss: 1.1833 - val_accuracy: 0.7155
Epoch 7/30
8982/8982 [==============================] - 3s 387us

# RNN model

In [29]:
from tensorflow.keras.layers import LSTM, GRU

In [34]:
model = Sequential()
model.add( Embedding( 400 , embeddings_dim , input_length=200)  )
# model.add( Dropout(0.2) )

model.add( LSTM(250, return_sequences=True) )
model.add( LSTM(300) )

model.add( Dense(46, activation='softmax') )

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 200, 200)          80000     
_________________________________________________________________
lstm_4 (LSTM)                (None, 200, 250)          451000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 300)               661200    
_________________________________________________________________
dense_12 (Dense)             (None, 46)                13846     
Total params: 1,206,046
Trainable params: 1,206,046
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.fit(x_train_padded, y_train, batch_size=32, epochs=30, verbose=1, validation_data=(x_test_padded, y_test) )

Train on 8982 samples, validate on 2246 samples
Epoch 1/30
8982/8982 [==============================] - 10s 1ms/sample - loss: 2.0916 - accuracy: 0.4582 - val_loss: 1.8863 - val_accuracy: 0.5209
Epoch 2/30
8982/8982 [==============================] - 8s 942us/sample - loss: 1.7918 - accuracy: 0.5429 - val_loss: 1.7335 - val_accuracy: 0.5623
Epoch 3/30
8982/8982 [==============================] - 8s 944us/sample - loss: 1.8236 - accuracy: 0.5314 - val_loss: 1.9186 - val_accuracy: 0.5129
Epoch 4/30
8982/8982 [==============================] - 8s 942us/sample - loss: 1.7274 - accuracy: 0.5596 - val_loss: 1.6884 - val_accuracy: 0.5748
Epoch 5/30
8982/8982 [==============================] - 8s 946us/sample - loss: 1.5880 - accuracy: 0.5910 - val_loss: 1.6663 - val_accuracy: 0.5797
Epoch 6/30
8982/8982 [==============================] - 8s 940us/sample - loss: 1.5189 - accuracy: 0.6129 - val_loss: 1.5775 - val_accuracy: 0.5984
Epoch 7/30
8982/8982 [==============================] - 8s 941us/